<a href="https://colab.research.google.com/github/joyceee916/ATMS-597-SP-2020-Project-5/blob/master/Project5_Joyce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 5 - Classification (version 1.0)
Tasks: Use supervised classification techniques to predict frozen vs. liquid precipitation type (i.e., when it is not raining, so rain and freezing rain vs. other categories such as snow, sleet, etc.) at the surface in ASOS weather observations.

Model 1: Logistic Regression classification with default values (baseline)

>>> from sklearn.linear_model import LogisticRegression
...
>>> clf = LogisticRegression(random_state=0).fit(X, y)
Model 2: Your pick!

Data: You will use 5-minute Automated Surface Observation System data available from NCDC (ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/). Data from 2000-2020 is available. Use a 70-30 random split to perform training and validation.

You can use Temperature, Dewpoint, Wind Direction, Wind Speed, and other derived variables from those data, or synthetic variables such as solar zenith angle, time of day, etc.

Each group will have a separate Midwest US city to examine:

Group D: KASX (Ashland, WI) 

The group with the improvement of values of Briar Skill Score for their Model 2 over the baseline SVM model will be given a prize.

Course presentations will be Thursday, April 23. A colab notebook or jupyter notebook can be submitted for the assignment.


In [10]:
pwd

'/content'

In [13]:
cd /content/drive/My Drive/ATMS 597/Project5

/content/drive/My Drive/ATMS 597/Project5


In [16]:
!pip install metpy
!pip install metar

     |████████████████████████████████| 327kB 2.7MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 194kB 48.7MB/s 
     |████████████████████████████████| 215kB 2.7MB/s 
  Created wheel for metar: filename=metar-1.7.0-cp36-none-any.whl size=198905 sha256=973d56721006edb2d94f9b671052b2c4ccd31a2bb9922f14c646e00167f1a392
  Stored in directory: /root/.cache/pip/wheels/8e/7b/5c/abc5749d1a4d2241b9dc824f6723f699f6eb2c8e6deeb1c294
Successfully built metar


In [17]:
from ftplib import FTP 
import pandas as pd
import metpy
import metpy.io

Cannot import USCOUNTIES and USSTATES without Cartopy installed.


In [18]:
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login(user='ftp', passwd='joycey2@illinois.gov') # substitute with your own email

'230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n ** This is a United States Department of Commerce computer     **\n ** system, which may be accessed and used only for             **\n ** official Government business by authorized personnel.       **\n ** Unauthorized access or use of this computer system may      **\n ** subject violators to criminal, civil, and/or administrative **\n ** action.  All information on this computer system may be     **\n ** intercepted, recorded, read, copied, and disclosed by and   **\n ** to authorized personnel for official purposes, including    **\n ** criminal investigations.  Access or use of this computer    **\n ** system by any person, whether authorized or unauthorized,   **\n ** constitutes consent to these terms.                         **\n ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n230 Anonymous access granted, restrictions apply'

In [0]:
# File Name Generator 
def folderNames(year):
    folder_prefix = '/pub/data/asos-fivemin/6401-'
    folder = folder_prefix + str(year) + '/'

    file_prefix = '64010KGRB'
    file_suffix = '.dat'
    file_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    files = [file_prefix+str(year)+month+file_suffix for month in file_month]

    return folder, files

In [0]:
def grabFile():
    #ftp.cwd('/pub/data/asos-fivemin/6401-2000/')

    #filename = '64010KGRB200001.dat'
    for year in range(2000, 2021):
        folder, files = folderNames(year)
        print(folder)
        try:
            ftp = FTP('ftp.ncdc.noaa.gov')
            ftp.login(user='ftp', passwd='joycey2@illinois.gov')
            ftp.cwd(folder)
        except: 
            print('couldnt log in or change to correct directory')
        for file in files:
            try: 
                localfile = open(file, 'wb')
                ftp.retrbinary('RETR ' + file, localfile.write, 1024)
                localfile.close()
            except: 
                return

In [0]:
grabFile()

/pub/data/asos-fivemin/6401-2000/
/pub/data/asos-fivemin/6401-2001/
/pub/data/asos-fivemin/6401-2002/
/pub/data/asos-fivemin/6401-2003/
/pub/data/asos-fivemin/6401-2004/
/pub/data/asos-fivemin/6401-2005/
/pub/data/asos-fivemin/6401-2006/
/pub/data/asos-fivemin/6401-2007/
/pub/data/asos-fivemin/6401-2008/
/pub/data/asos-fivemin/6401-2009/
/pub/data/asos-fivemin/6401-2010/
/pub/data/asos-fivemin/6401-2011/
/pub/data/asos-fivemin/6401-2012/
/pub/data/asos-fivemin/6401-2013/
/pub/data/asos-fivemin/6401-2014/
/pub/data/asos-fivemin/6401-2015/
/pub/data/asos-fivemin/6401-2016/
/pub/data/asos-fivemin/6401-2017/
/pub/data/asos-fivemin/6401-2018/
/pub/data/asos-fivemin/6401-2019/
/pub/data/asos-fivemin/6401-2020/


In [0]:
# Courtesy of Max Grover 
from datetime import datetime
import pandas as pd
from metpy.io import parse_metar_to_dataframe
import glob


def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC
    Input:
    --------
    file = Text file downloaded from NCDC
    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)
    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])

    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()

In [0]:
all_files = []
for year in range(2000, 2021):
    folder, files = folderNames(year)
    all_files.append(files) # creates a list of lists 

In [37]:
print(all_files)

[['64010KGRB200001.dat', '64010KGRB200002.dat', '64010KGRB200003.dat', '64010KGRB200004.dat', '64010KGRB200005.dat', '64010KGRB200006.dat', '64010KGRB200007.dat', '64010KGRB200008.dat', '64010KGRB200009.dat', '64010KGRB200010.dat', '64010KGRB200011.dat', '64010KGRB200012.dat'], ['64010KGRB200101.dat', '64010KGRB200102.dat', '64010KGRB200103.dat', '64010KGRB200104.dat', '64010KGRB200105.dat', '64010KGRB200106.dat', '64010KGRB200107.dat', '64010KGRB200108.dat', '64010KGRB200109.dat', '64010KGRB200110.dat', '64010KGRB200111.dat', '64010KGRB200112.dat'], ['64010KGRB200201.dat', '64010KGRB200202.dat', '64010KGRB200203.dat', '64010KGRB200204.dat', '64010KGRB200205.dat', '64010KGRB200206.dat', '64010KGRB200207.dat', '64010KGRB200208.dat', '64010KGRB200209.dat', '64010KGRB200210.dat', '64010KGRB200211.dat', '64010KGRB200212.dat'], ['64010KGRB200301.dat', '64010KGRB200302.dat', '64010KGRB200303.dat', '64010KGRB200304.dat', '64010KGRB200305.dat', '64010KGRB200306.dat', '64010KGRB200307.dat', '64

In [0]:
test = [['64010KGRB200001.dat', '64010KGRB200002.dat'], ['64010KGRB200003.dat', '64010KGRB200004.dat']]

In [33]:
for list in test:
    for file in list: 
        parse_metar_file(file)

21

## Old Code 

In [0]:
with open('64010KGRB200001.dat', 'r') as fh:
    lines = fh.readlines()

# Remove newlines, tabs, and split each string separated by spaces.
clean = [line.replace('\t', '').split() for line in lines]

# Feed the data into a DataFrame.
data = pd.DataFrame(clean[:])

In [0]:
#!/usr/bin/python

import sys
import ftplib
import os
import time

server = "ftp.ncdc.noaa.gov"
user = "ftp"
password = "joycey2@illinois.edu"
source = "/pub/data/asos-fivemin/6401-2000/"
destination = "/content/drive/My Drive/ATMS 597/Project5"
interval = 0.05

ftp = ftplib.FTP(server)
ftp.login(user, password)



'230-****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n ** This is a United States Department of Commerce computer     **\n ** system, which may be accessed and used only for             **\n ** official Government business by authorized personnel.       **\n ** Unauthorized access or use of this computer system may      **\n ** subject violators to criminal, civil, and/or administrative **\n ** action.  All information on this computer system may be     **\n ** intercepted, recorded, read, copied, and disclosed by and   **\n ** to authorized personnel for official purposes, including    **\n ** criminal investigations.  Access or use of this computer    **\n ** system by any person, whether authorized or unauthorized,   **\n ** constitutes consent to these terms.                         **\n ****** WARNING ** WARNING ** WARNING ** WARNING ** WARNING ******\n230 Anonymous access granted, restrictions apply'

In [0]:
source = "/pub/data/asos-fivemin/6401-2000/"
destination = "/content/drive/My Drive/ATMS 597/Project5"
mkdir_p(destination[0:len(destination)-1] + source)

NameError: ignored

In [0]:
def downloadFiles(path, destination):
    try:
        ftp.cwd(path)       
        os.chdir(destination)
        mkdir_p(destination[0:len(destination)-1] + path)
        print("Create d: " + destination[0:len(destination)-1] + path)
    except OSError:     
        pass
    except ftplib.error_perm:       
        print("Error: could not change to " + path)
        sys.exit("Ending Application")
    
    filelist=ftp.nlst()

    for file in filelist:
        print(file)
        """time.sleep(interval)
        try:            
            ftp.cwd(path + file + "/")          
            downloadFiles(path + file + "/", destination)
        except ftplib.error_perm:
            os.chdir(destination[0:len(destination)-1] + path)
            
            try:
                ftp.retrbinary("RETR " + file, open(os.path.join(destination + path, file),"wb").write)
                print("Downloaded: " + file)
            except:
                print("Error: File could not be downloaded " + file)"""
    return
    


downloadFiles(source, destination)

NameError: ignored

In [0]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [0]:
ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/


Old Code

In [0]:
from __future__ import print_function
import json
import time
import datetime

In [0]:
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"

In [0]:
def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""

In [0]:
def get_stations_from_filelist(filename):
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return stations


In [0]:
def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = []
    states = """AK AL AR AZ CA CO CT DE FL GA HI IA ID IL IN KS KY LA MA MD ME
     MI MN MO MS MT NC ND NE NH NJ NM NV NY OH OK OR PA RI SC SD TN TX UT VA VT
     WA WI WV WY"""
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ["AWOS"]
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = (
            "https://mesonet.agron.iastate.edu/" "geojson/network/%s.geojson"
        ) % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict["features"]:
            stations.append(site["properties"]["sid"])
    return stations

In [0]:
def main():
    """Our main method"""
    # timestamps in UTC to request data for
    startts = datetime.datetime(2012, 8, 1)
    endts = datetime.datetime(2012, 9, 1)

    service = SERVICE + "data=all&tz=Etc/UTC&format=comma&latlon=yes&"

    service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
    service += endts.strftime("year2=%Y&month2=%m&day2=%d&")

    # Two examples of how to specify a list of stations
    stations = get_stations_from_networks()
    # stations = get_stations_from_filelist("mystations.txt")
    for station in stations:
        uri = "%s&station=%s" % (service, station)
        print("Downloading: %s" % (station,))
        data = download_data(uri)
        outfn = "%s_%s_%s.txt" % (
            station,
            startts.strftime("%Y%m%d%H%M"),
            endts.strftime("%Y%m%d%H%M"),
        )
        out = open(outfn, "w")
        out.write(data)
        out.close()


if __name__ == "__main__":
    main()

Downloading: AXA
Downloading: IKV
Downloading: AIO
Downloading: ADU
Downloading: BNW
Downloading: CIN
Downloading: TVK
Downloading: CNC
Downloading: CCY
Downloading: CKP
Downloading: ICL
Downloading: CAV
Downloading: CWI
Downloading: CBF
Downloading: CSQ
Downloading: DEH
Downloading: DNS
Downloading: FFL
Downloading: FXY
Downloading: FOD
Downloading: FSW
Downloading: GGI
Downloading: HPT
Downloading: HNR
Downloading: IIB
Downloading: IFA
Downloading: EOK
Downloading: OXV
Downloading: LRJ
Downloading: MXO
Downloading: MPZ
Downloading: MUT
Downloading: TNU
Downloading: OLZ
Downloading: ORC
Downloading: I75
Downloading: OOA
Downloading: PEA
Downloading: PRO
Downloading: RDK
Downloading: SHL
Downloading: SDA
Downloading: SXK
Downloading: SLB
Downloading: VTI
Downloading: AWG
Downloading: EBS
Downloading: PADK
Downloading: PAKH
Downloading: PAUT
Downloading: PALP
Downloading: PAFM
Downloading: PAKP
Downloading: PANC
Downloading: PAGN
Downloading: PANI
Downloading: PANT
Downloading: PANV
Dow

In [0]:
metpy.io.parse_metar_to_dataframe(metar_text, year=2020, month=1)